In [1]:
%matplotlib inline
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [2]:
## spark Context 
sc = SparkContext( 'local', 'pyspark')

In [3]:
sqlContext = SQLContext(sc)

In [4]:
def parse_line(l):
    try:
        return l.split(",")
    except:
        print("error in processing {0}".format(l))

In [5]:
songs = sc.textFile('/Users/jbabcock/Downloads/YearPredictionMSD.txt').\
map(lambda x : parse_line(x)).\
toDF()

In [7]:
from pyspark.mllib.regression import LabeledPoint
songs_labeled = songs.map( lambda x: LabeledPoint(x[0],x[1:]) )

In [8]:
## splitting train

songs_train = songs_labeled.zipWithIndex().\
filter( lambda x: x[1] < 463715).\
map( lambda x: x[0] )

In [9]:
## splitting test

songs_test = songs_labeled.zipWithIndex().\
filter( lambda x: x[1] >= 463715).\
map( lambda x: x[0] )

In [18]:
from pyspark.mllib.tree import RandomForest
rf = RandomForest.trainRegressor(songs_train,{},50,"auto","variance",10,32)

In [19]:
prediction = rf.predict(songs_test.map(lambda x: x.features))
predictedObserved = songs_test.map(lambda lp: lp.label).zip(prediction)

In [20]:
from pyspark.mllib.evaluation import RegressionMetrics

In [21]:
RegressionMetrics(predictedObserved).r2

-3.69830379609341

In [22]:
predictedObserved.take(10)

[(2007.0, 1995.9064460339775),
 (2003.0, 2002.534486616815),
 (2005.0, 1999.4269553417332),
 (2003.0, 2002.2919078641394),
 (2005.0, 2001.720244097928),
 (2007.0, 1997.7955438534639),
 (2003.0, 2000.5908110171194),
 (2003.0, 1999.874075396483),
 (2003.0, 1997.9664817235432),
 (2005.0, 2001.4931432241076)]